In [ ]:
# Import necessary libraries
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from dash.dash_table import DataTable
import dash_leaflet as dl
from jupyter_dash import JupyterDash
from crud import AnimalShelter

# Initialize Dash app
app = JupyterDash(__name__)

# Set up MongoDB connection
username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)
df = pd.DataFrame.from_records(shelter.read({}))  # Retrieve all data from MongoDB
df.drop(columns=['_id'], inplace=True)  # Remove '_id' column

# Dashboard layout
app.layout = html.Div([
    html.H1("Grazioso Salvare Dashboard"),
    html.Div([
        html.Label('Select Rescue Type'),
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[{'label': i, 'value': i} for i in df['rescue_type'].unique()],
            value='Water Rescue'
        ),
        html.Label('Select Dog Breed'),
        dcc.Dropdown(
            id='breed-dropdown',
            options=[{'label': i, 'value': i} for i in df['breed'].unique()],
            value='Labrador'
        ),
    ]),
    html.Br(),
    DataTable(
        id='animal-data-table',
        columns=[{'name': i, 'id': i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Pagination for the table
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px', 'fontSize': '12px'},
        row_selectable="single",  # Single row selection
        selected_rows=[0],  # Default first row selected
        sort_action="native",  # Sorting enabled
        filter_action="native",  # Filtering enabled
    ),
    html.Br(),
    dcc.Graph(id='geolocation-chart'),
    html.Br(),
    html.Div(id='map-id', className='col s12 m6')  # Map container
])

# Callbacks to update data table and geolocation map based on user input
@app.callback(
    [Output('animal-data-table', 'data'),
     Output('geolocation-chart', 'figure'),
     Output('map-id', 'children')],
    [Input('rescue-type-dropdown', 'value'),
     Input('breed-dropdown', 'value')]
)
def update_data(rescue_type, breed):
    filtered_data = df[(df['rescue_type'] == rescue_type) & (df['breed'] == breed)]
    
    # Update Data Table
    data = filtered_data.to_dict('records')
    
    # Create geolocation chart using Plotly
    geo_fig = px.scatter_geo(
        filtered_data,
        lat='location_lat',
        lon='location_long',
        hover_name="animal_id",
        title="Animal Locations"
    )
    
    # Create Map using Dash-Leaflet
    map_children = dl.Map(style={'width': '1000px', 'height': '500px'}, 
                          center=[30.75, -97.48], zoom=10, children=[
                              dl.TileLayer(id="base-layer-id"),
                              dl.Marker(position=[filtered_data.iloc[0]['location_lat'], 
                                                  filtered_data.iloc[0]['location_long']],
                                        children=[dl.Tooltip(filtered_data.iloc[0]['animal_id'])])
                          ])
    
    return data, geo_fig, map_children

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
